# AgentOps Langchain Agent Implementation

Using AgentOps monitoring with Langchain is simple. We've created a LangchainCallbackHandler that will do all of the heavy lifting!

First we'll import the typical Langchain packages:

In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from dotenv import load_dotenv
from langchain.agents import tool

The only difference with using AgentOps is that we'll also import this special Callback Handler

In [2]:
from agentops.langchain_callback_handler import LangchainCallbackHandler

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [3]:
load_dotenv()

False

This is where AgentOps comes into play. Before creating our LLM instance via Langchain, first we'll create an instance of the AO LangchainCallbackHandler. After the handler is initialiezd, a session will be recorded automatically.

Pass in your API key, and optionally any tags to describe this session for easier lookup in the AO dashboard. You can also retrieve the `session_id` of the newly created session.

In [4]:
AGENTOPS_API_KEY = os.environ.get('AGENTOPS_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

handler = LangchainCallbackHandler(api_key=AGENTOPS_API_KEY, tags=['Langchain Example'])

print("Agent Ops session ID: " + handler.session_id)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model='gpt-3.5-turbo')

Agent Ops session ID: 80c6593c-ae90-4b48-98e1-d17fcd801c70


Agents generally use tools. Let's define a simple tool here. Tool usage is also recorded.

In [5]:
@tool
def find_movie(term) -> str:
    """Find available movies"""
    return 'Citizen Kane'


tools = [find_movie]

Finally, let's use our agent! Pass in the callback handler to the agent, and all the actions will be recorded in the AO Dashboard

In [6]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         callbacks=[handler], # You must pass in a callback handler to record your agent
                         handle_parsing_errors=True)

In [7]:
agent.run("What movies are playing?", callbacks=[handler])



> Entering new AgentExecutor chain...
Could not parse LLM output: I can use the `find_movie` tool to find the available movies.
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: I will use the `find_movie` tool to find the available movies.
Observation: Invalid or incomplete response
Thought:I will use the `find_movie` tool to find the available movies.
Action:
```
{
  "action": "find_movie",
  "action_input": ""
}
```
Observation: Citizen Kane
Thought:Could not parse LLM output: The movie "Citizen Kane" is currently playing.
Observation: Invalid or incomplete response
Thought:I will use the `find_movie` tool to find the available movies.
Action:
```
{
  "action": "find_movie",
  "action_input": ""
}
```
Observation: Citizen Kane
Thought:I will use the `find_movie` tool to find the available movies.
Action:
```
{
  "action": "find_movie",
  "action_input": ""
}
```

Observation: Citizen Kane
Thought:Could not parse LLM output: I now know the final answer

'The movie "Citizen Kane" is currently playing.'